# Generatívne siete

Rekurentné neurónové siete (RNN) a ich varianty s bránami, ako sú bunky Long Short Term Memory (LSTM) a Gated Recurrent Units (GRU), poskytujú mechanizmus na modelovanie jazyka, t.j. dokážu sa naučiť poradie slov a poskytovať predpovede pre nasledujúce slovo v sekvencii. To nám umožňuje používať RNN na **generatívne úlohy**, ako je bežná generácia textu, strojový preklad a dokonca aj popisovanie obrázkov.

V architektúre RNN, ktorú sme diskutovali v predchádzajúcej jednotke, každá jednotka RNN produkovala ako výstup nasledujúci skrytý stav. Avšak, môžeme tiež pridať ďalší výstup ku každej rekurentnej jednotke, čo nám umožní generovať **sekvenciu** (ktorá má rovnakú dĺžku ako pôvodná sekvencia). Navyše, môžeme použiť jednotky RNN, ktoré neprijímajú vstup na každom kroku, ale len berú nejaký počiatočný stavový vektor a následne produkujú sekvenciu výstupov.

V tomto notebooku sa zameriame na jednoduché generatívne modely, ktoré nám pomáhajú generovať text. Pre jednoduchosť si vytvoríme **sieť na úrovni znakov**, ktorá generuje text písmeno po písmene. Počas tréningu musíme vziať nejaký textový korpus a rozdeliť ho na sekvencie písmen.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Vytváranie slovníka znakov

Na vytvorenie generatívnej siete na úrovni znakov je potrebné rozdeliť text na jednotlivé znaky namiesto slov. To sa dá dosiahnuť definovaním iného tokenizéra:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Pozrime sa na príklad, ako môžeme zakódovať text z nášho datasetu:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Trénovanie generatívnej RNN

Spôsob, akým budeme trénovať RNN na generovanie textu, je nasledovný. V každom kroku vezmeme sekvenciu znakov s dĺžkou `nchars` a požiadame sieť, aby pre každý vstupný znak vygenerovala nasledujúci výstupný znak:

![Obrázok zobrazujúci príklad generovania slova 'HELLO' pomocou RNN.](../../../../../translated_images/sk/rnn-generate.56c54afb52f9781d.webp)

V závislosti od konkrétneho scenára môžeme chcieť zahrnúť aj špeciálne znaky, ako napríklad *koniec sekvencie* `<eos>`. V našom prípade chceme sieť trénovať na nekonečné generovanie textu, preto nastavíme veľkosť každej sekvencie na `nchars` tokenov. Každý tréningový príklad tak bude pozostávať z `nchars` vstupov a `nchars` výstupov (čo je vstupná sekvencia posunutá o jeden symbol doľava). Minibatch bude obsahovať niekoľko takýchto sekvencií.

Spôsob, akým budeme generovať minibatch, je nasledovný: vezmeme každý text správy s dĺžkou `l` a z neho vygenerujeme všetky možné kombinácie vstup-výstup (bude ich `l-nchars`). Tieto kombinácie vytvoria jeden minibatch, pričom veľkosť minibatchov bude pri každom tréningovom kroku odlišná.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Teraz si definujme generátorovú sieť. Môže byť založená na akejkoľvek rekurentnej bunke, o ktorej sme hovorili v predchádzajúcej jednotke (jednoduchá, LSTM alebo GRU). V našom príklade použijeme LSTM.

Keďže sieť prijíma znaky ako vstup a veľkosť slovníka je pomerne malá, nepotrebujeme vrstvu na vkladanie (embedding layer), jednohotovo zakódovaný vstup môže ísť priamo do LSTM bunky. Avšak, pretože ako vstup odovzdávame čísla znakov, musíme ich pred odovzdaním do LSTM jednohotovo zakódovať. Toto sa vykonáva volaním funkcie `one_hot` počas prechodu `forward`. Výstupný enkodér bude lineárna vrstva, ktorá prevedie skrytý stav na jednohotovo zakódovaný výstup.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Počas tréningu chceme byť schopní vzorkovať generovaný text. Na tento účel definujeme funkciu `generate`, ktorá vytvorí výstupný reťazec dĺžky `size`, začínajúc od počiatočného reťazca `start`.

Funguje to nasledovne. Najprv prejdeme celý počiatočný reťazec cez sieť a získame výstupný stav `s` a ďalší predpovedaný znak `out`. Keďže `out` je zakódovaný v one-hot formáte, použijeme `argmax`, aby sme získali index znaku `nc` vo slovníku, a pomocou `itos` zistíme skutočný znak, ktorý pridáme do výsledného zoznamu znakov `chars`. Tento proces generovania jedného znaku sa opakuje `size` krát, aby sme vygenerovali požadovaný počet znakov.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Poďme na tréning! Tréningová slučka je takmer rovnaká ako vo všetkých našich predchádzajúcich príkladoch, ale namiesto presnosti vypisujeme každých 1000 epoch vygenerovaný text.

Osobitnú pozornosť treba venovať spôsobu, akým počítame stratu. Stratu musíme vypočítať na základe výstupu zakódovaného v one-hot formáte `out` a očakávaného textu `text_out`, čo je zoznam indexov znakov. Našťastie, funkcia `cross_entropy` očakáva ako prvý argument nenormalizovaný výstup siete a ako druhý číslo triedy, čo je presne to, čo máme. Táto funkcia tiež automaticky vykonáva priemerovanie podľa veľkosti minibatchu.

Tréning tiež obmedzujeme na počet vzoriek `samples_to_train`, aby sme nemuseli čakať príliš dlho. Odporúčame vám experimentovať a skúsiť dlhší tréning, prípadne na niekoľko epoch (v takom prípade by ste museli vytvoriť ďalšiu slučku okolo tohto kódu).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Tento príklad už generuje pomerne dobrý text, ale dá sa ešte vylepšiť niekoľkými spôsobmi:  
* **Lepšia generácia minibatchov**. Spôsob, akým sme pripravili dáta na trénovanie, bol taký, že sme generovali jeden minibatch z jednej vzorky. To nie je ideálne, pretože minibatche majú rôzne veľkosti a niektoré z nich sa ani nedajú vygenerovať, pretože text je menší ako `nchars`. Navyše, malé minibatche nedostatočne zaťažujú GPU. Múdrejšie by bolo získať jeden veľký blok textu zo všetkých vzoriek, potom vygenerovať všetky páry vstup-výstup, zamiešať ich a vytvoriť minibatche rovnakej veľkosti.  
* **Viacvrstvové LSTM**. Má zmysel vyskúšať 2 alebo 3 vrstvy LSTM buniek. Ako sme spomenuli v predchádzajúcej jednotke, každá vrstva LSTM extrahuje určité vzory z textu, a v prípade generátora na úrovni znakov môžeme očakávať, že nižšia úroveň LSTM bude zodpovedná za extrakciu slabík, a vyššie úrovne - za slová a kombinácie slov. Toto sa dá jednoducho implementovať odovzdaním parametra počtu vrstiev do konštruktora LSTM.  
* Môžete tiež experimentovať s **GRU jednotkami** a zistiť, ktoré z nich fungujú lepšie, a s **rôznymi veľkosťami skrytých vrstiev**. Príliš veľká skrytá vrstva môže viesť k overfittingu (napr. sieť sa naučí presný text), a menšia veľkosť nemusí priniesť dobrý výsledok.  


## Generovanie mäkkého textu a teplota

V predchádzajúcej definícii `generate` sme vždy vyberali znak s najvyššou pravdepodobnosťou ako ďalší znak v generovanom texte. To malo za následok, že text často "cykloval" medzi rovnakými sekvenciami znakov znova a znova, ako v tomto príklade:
```
today of the second the company and a second the company ...
```

Ak sa však pozrieme na rozdelenie pravdepodobností pre ďalší znak, môže sa stať, že rozdiel medzi niekoľkými najvyššími pravdepodobnosťami nie je veľký, napríklad jeden znak môže mať pravdepodobnosť 0.2, iný 0.19, atď. Napríklad, pri hľadaní ďalšieho znaku v sekvencii '*play*' môže byť ďalší znak rovnako dobre medzera alebo **e** (ako v slove *player*).

To nás vedie k záveru, že nie je vždy "spravodlivé" vybrať znak s vyššou pravdepodobnosťou, pretože výber druhého najvyššieho môže stále viesť k zmysluplnému textu. Je rozumnejšie **vzorkovať** znaky z rozdelenia pravdepodobností, ktoré poskytuje výstup siete.

Toto vzorkovanie sa dá vykonať pomocou funkcie `multinomial`, ktorá implementuje tzv. **multinomiálne rozdelenie**. Funkcia, ktorá implementuje toto **mäkké** generovanie textu, je definovaná nižšie:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Zaviedli sme ďalší parameter nazývaný **teplota**, ktorý sa používa na určenie, ako striktne by sme sa mali držať najvyššej pravdepodobnosti. Ak je teplota 1.0, vykonávame spravodlivé multinomiálne vzorkovanie, a keď teplota dosiahne nekonečno - všetky pravdepodobnosti sa stanú rovnakými a náhodne vyberáme ďalší znak. V príklade nižšie môžeme pozorovať, že text sa stáva nezmyselným, keď príliš zvýšime teplotu, a pripomína „cyklický“ ťažko generovaný text, keď sa teplota blíži k 0.



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre dôležité informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
